In [18]:
import pandas as pd
import numpy as np
import nibabel as nib
import datetime
import os

In [40]:
# Parameters
toy = False
run_all = True
path = 'home/ubuntu/fmriprep/'
output_dir = '../output/'
input_dir = '/home/ubuntu/brain_age/drsulliv3/'

n_jobs = 20 #amount of cores
cv=4
description = 'CNP_func_gridsearch_age_'
log_file = description+datetime.datetime.now().strftime("%y-%m-%d-%H-%M-%S")

In [29]:
log_file

'CNP_func_gridsearch_age_18-08-09-21-44-25'

In [30]:
# Load subject ids
if run_all:
    files = os.listdir(path)
    subj = [n[4:9] for n in files if n.startswith('sub')] #here we will filter the file names to leave only subj number as bellow

    file = 'subjlist_wfiles_CNP.txt'
    with open(input_dir+file, 'r') as f:
        lines = f.readlines()

#np.load(/home/ubuntu/brain_age/drsulliv3/)
# This was done once before on the T1. This order was randomized. 
subj =  ['10672', '10940', '10361', '10719', '11112', '10356', '11106',
              '11030', '10340', '10570', '10329', '10871', '10746', '10325',
              '10987', '10724', '11105', '10492', '10429', '10565', '10977',
              '10316', '10460', '10678', '10704', '11143', '11067', '10274',
              '10304', '10696', '11098', '10998', '10455', '10629', '10963',
              '10345', '11019', '10271', '11108', '10225', '11149', '11050',
              '10448', '10680', '11068', '11104', '10388', '10530', '10206',
              '10692', '10438', '10228', '10762', '10708', '10235', '10788',
              '10527', '10631', '11044', '10844', '10517', '11090', '10377',
              '10934', '10575', '10471', '10159', '10189', '10779', '10949',
              '11052', '11059', '10280', '11156', '11077', '10975', '10290',
              '10557', '11066', '10686', '10891', '10525', '11061', '10321',
              '10697', '10668', '10171', '11122', '10217', '10785', '10339',
              '10523', '10506', '10347', '10707', '11062', '10958', '10376',
              '10893', '10440', '10968', '10365', '10292', '11142', '10478',
              '11088', '10912', '10855', '10487', '10273', '11097', '10638',
              '11131', '10227', '10269', '10882', '10877', '10524', '10624',
              '10249', '11128', '10674']


In [31]:
len(subj)

122

In [41]:
# Load volumes for each subject into a dictionary    
if run_all:
    filenames = []
    for i in range(len(subj)):
        filenames.append(path+'sub-'+subj[i]+'/func/sub-'+subj[i]+'_task-rest_bold_space-MNI152NLin2009cAsym_preproc.nii.gz')
    cofounds = []
    for i in range(len(subj)):
        cofounds.append(path+'sub-'+subj[i]+'/func/sub-'+subj[i]+'_task-rest_bold_confounds.tsv')



In [42]:
filenames

['home/ubuntu/fmriprep/sub-10672/func/sub-10672_task-rest_bold_space-MNI152NLin2009cAsym_preproc.nii.gz',
 'home/ubuntu/fmriprep/sub-10940/func/sub-10940_task-rest_bold_space-MNI152NLin2009cAsym_preproc.nii.gz',
 'home/ubuntu/fmriprep/sub-10361/func/sub-10361_task-rest_bold_space-MNI152NLin2009cAsym_preproc.nii.gz',
 'home/ubuntu/fmriprep/sub-10719/func/sub-10719_task-rest_bold_space-MNI152NLin2009cAsym_preproc.nii.gz',
 'home/ubuntu/fmriprep/sub-11112/func/sub-11112_task-rest_bold_space-MNI152NLin2009cAsym_preproc.nii.gz',
 'home/ubuntu/fmriprep/sub-10356/func/sub-10356_task-rest_bold_space-MNI152NLin2009cAsym_preproc.nii.gz',
 'home/ubuntu/fmriprep/sub-11106/func/sub-11106_task-rest_bold_space-MNI152NLin2009cAsym_preproc.nii.gz',
 'home/ubuntu/fmriprep/sub-11030/func/sub-11030_task-rest_bold_space-MNI152NLin2009cAsym_preproc.nii.gz',
 'home/ubuntu/fmriprep/sub-10340/func/sub-10340_task-rest_bold_space-MNI152NLin2009cAsym_preproc.nii.gz',
 'home/ubuntu/fmriprep/sub-10570/func/sub-1057

In [34]:
# if run_all:
#     path_and_filenames = [path+n for n in filenames]

#     nifti_masker = NiftiMasker(
#         standardize=False,
#         smoothing_fwhm=2, mask_strategy='epi',
#         memory='nilearn_cache')  # cache options
#     gm_maps_masked = nifti_masker.fit_transform(filenames)
#     n_samples, n_features = gm_maps_masked.shape
#     print("%d samples, %d features" % (n_samples, n_features))

In [36]:
from nilearn import datasets

parcellations = datasets.fetch_atlas_basc_multiscale_2015(version='sym')

# We show here networks of 64, 197, 444
networks_64 = parcellations['scale064']
networks_197 = parcellations['scale197']
networks_444 = parcellations['scale444']

/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [43]:
from nilearn.input_data import NiftiLabelsMasker

def func_2_corr_matrix(parcellation, name):
    masker = NiftiLabelsMasker(labels_img=parcellation, standardize=True,
                               memory='nilearn_cache', verbose=5)
    #n_regions = 197
    from nilearn.connectome import ConnectivityMeasure, sym_matrix_to_vec
    correlations = []
    upper_triang_corrs = []
    # Initializing ConnectivityMeasure object with kind='correlation'
    connectome_measure = ConnectivityMeasure(kind='correlation')
    for filename in filenames:
        timeseries_each_subject = masker.fit_transform(filename)
        # call fit_transform from ConnectivityMeasure object
        correlation = connectome_measure.fit_transform([timeseries_each_subject])
        # saving each subject correlation to correlations
        correlations.append(correlation)
        # get the upper triangular correlation matrix of correlation
        upper_triang_corr = sym_matrix_to_vec(correlation, discard_diagonal=False)
        # saving each subject upper_triang_corr to upper_triang_corrs
        upper_triang_corrs.append(upper_triang_corr) 
    #   Split train test sets and save
    X_train = gm_maps_masked[:84]
    X_test = gm_maps_masked[84:]
    y_train = y_age[:84]
    y_test = y_age[84:]
    random_subj_train = subj[:84]
    random_subj_test = subj[84:]
    np.savez_compressed(output_dir+log_file+'CNP_func'+name+'_train_set',a=X_train, b=y_train, c=random_subj_train)
    np.savez_compressed(output_dir+log_file+'CNP_func'+name+'_test_set',a=X_test, b=y_test, c=random_subj_test)

func_2_corr_matrix(networks_197, '197')
func_2_corr_matrix(networks_444, '444')

[NiftiLabelsMasker.fit_transform] loading data from /home/ubuntu/nilearn_data/basc_multiscale_2015/template_cambridge_basc_multiscale_nii_sym/template_cambridge_basc_multiscale_sym_scale197.nii.gz


ValueError: File not found: 'home/ubuntu/fmriprep/sub-10672/func/sub-10672_task-rest_bold_space-MNI152NLin2009cAsym_preproc.nii.gz'

In [ ]:
if not run_all:
    loaded = np.load(output_dir+'train_set.npz')
    X_train, y_train, random_subj_train = loaded['a'], loaded['b'], loaded['c']